# Strands Agents を用いたエージェントのスウォーム作成

## Swarm マルチエージェントシステムの理解

Swarm は、複数のエージェントがチームとして協力して複雑なタスクを解決する協調型エージェントオーケストレーションシステムです。従来の逐次的または階層的なマルチエージェントシステムとは異なり、Swarm は共有コンテキストとワーキングメモリを持つエージェント間の自律的な調整を可能にします。

* **共有ワーキングメモリを持つ自己組織化エージェントチーム**
* **エージェント間のツールベースの調整**
* **中央制御なしの自律エージェント協力**
* **エージェント能力に基づく動的タスク配分**
* **共有コンテキストを通じた集団知能**
* **テキスト、画像、その他のコンテンツタイプを扱うマルチモーダル入力対応**

## Swarm の動作原理

Swarm は出現知能の原理で動作します - 専門化したエージェントのグループが協力することで単一のエージェントよりも効果的に問題を解決できるという考え方です。Swarm 内の各エージェントは：

1. 完全なタスクコンテキストにアクセス可能
2. どのエージェントがタスクに取り組んだかの履歴を確認可能
3. 他のエージェントが寄与した共有知識にアクセス可能
4. 異なる専門知識を持つ別のエージェントにいつ引き継ぐかを決定可能

## 1. Swarm ツールによるクイックスタート

Strands Agents SDK は、マルチエージェントシステムの実装を簡素化するビルトインの swarm ツールを提供し、ユーザーにクイックスタートを提供します。このツールは、専門化されたエージェント間の自律調整を持つ柔軟な swarm 知能システムを実装します。
    
この例では：
1. エージェントが swarm ツールを使用して、研究者、アナリスト、テクニカルライターを含む専門化されたエージェントのチームを動的に作成
2. 次にエージェントが swarm を実行
3. swarm エージェントが必要に応じて互いに引き継ぎながら自律的に協力
4. エージェントが swarm 結果を分析し、ユーザーに包括的な応答を提供

In [ ]:
!pip install -r requirements.txt

In [ ]:
from strands import Agentfrom strands_tools import swarm

## 1.1 ツールとしての Swarm

In [ ]:
# Agent initializationagent = Agent(model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",tools=[swarm])# Invocation through natural languageresult = str(agent(    "Use a swarm of 4 agents to analyze the current market trend for generative ai based agents."))

In [ ]:
print(result)

swarm ツールは、Strands SDK のネイティブ Swarm マルチエージェントパターンに基づく柔軟な swarm 知能システムを実装します。このシステムは、共有コンテキストとツールベースの調整を通じて自律的に協力するカスタム専門化 AI エージェントチームをユーザーが定義できるようにします。

**主な特徴：**
- **カスタムエージェントチーム**: 個別のシステムプロンプト、エージェントごとのツール設定、モデル設定を持つユーザー定義エージェント仕様
- **自律調整**: すべてのエージェントにわたる共有ワーキングメモリを持つビルトイン調整ツール (handoff_to_agent, complete_swarm_task)
- **高度な設定**: エージェントごとの個別モデルプロバイダと設定、カスタマイズ可能なツールアクセス、包括的なタイムアウトと安全機構
- **出現集団知能**: エージェントが自律的に協力または引き継ぎを決定し、能力に基づく動的タスク配分

swarm ツールの完全な実装は [Strands Tools リポジトリ](https://github.com/strands-agents/tools/blob/main/src/strands_tools/swarm.py) にあります。

**主なパラメータ：**
- `task`: swarm が処理するメインタスク
- `agents`: 名前、system_prompt、tools、model_provider、model_settings を持つエージェント仕様リスト
- `max_handoffs`: 許可されるエージェント引き継ぎ最大回数 (デフォルト: 20)
- `max_iterations`: すべてのエージェントにわたる最大総イテレーション数 (デフォルト: 20)
- `execution_timeout`: 総実行タイムアウト秒数 (デフォルト: 900.0)
- `node_timeout`: 個別エージェントタイムアウト秒数 (デフォルト: 300.0)
- `repetitive_handoff_detection_window`: ピンポン動作をチェックする最近のノード数
- `repetitive_handoff_min_unique_agents`: 最近のシーケンスで必要な最小ユニークノード数

**Swarm ツールの動作原理：**
1. **エージェント作成**: ユーザー仕様に基づく個別モデルプロバイダとツールを持つ専門化エージェントを作成
2. **Swarm 初期化**: 調整ツールと安全機構で Strands SDK Swarm を設定
3. **自律実行**: 中央制御なしでエージェントが引き継ぎと共有コンテキストを通じて協力
4. **結果集約**: 個別寄与を集め、包括的な実行メトリクスを提供

## 2. Strands Agents による Swarm 作成

Strands Agents SDK では、異なるモデルプロバイダを使用したり異なる設定を持つ既存の Agent オブジェクトを使用して swarm を作成できます。Swarm システムは、オートハンドオフ機構と共有コンテキストを通じてエージェント間の自律調整を可能にし、エージェントが必要とする専門知識を他のエージェントから動的に取得できるようにします。

### 2.1 オートハンドオフによるエージェント調整

<p align="center">
    <img src="./images/swarm_example.png">
</p>

この調整アプローチでは、エージェントが異なる専門知識が必要なときにビルトインのハンドオフツールを使用して他の専門化エージェントに自律的に制御を移転します。各エージェントには、`handoff_to_agent` や `complete_swarm_task` のような調整ツールが自動的に装備されます。以下の例は、共有コンテキストと自律ハンドオフを通じて協力する専門化エージェントの swarm を示します：

In [ ]:
# Create specialized agents with different expertiseresearch_agent = Agent(system_prompt="""You are a Research Agent specializing in gathering and analyzing information.\nYour role in the swarm is to provide factual information and research insights on the topic.\nYou should focus on providing accurate data and identifying key aspects of the problem.\nWhen receiving input from other agents, evaluate if their information aligns with your research.\n""", \nname="research_agent",model="us.anthropic.claude-3-7-sonnet-20250219-v1:0")creative_agent = Agent(system_prompt="""You are a Creative Agent specializing in generating innovative solutions.\nYour role in the swarm is to think outside the box and propose creative approaches.\nYou should build upon information from other agents while adding your unique creative perspective.\nFocus on novel approaches that others might not have considered.\n""", \nname="creative_agent",model="us.anthropic.claude-3-7-sonnet-20250219-v1:0")critical_agent = Agent(system_prompt="""You are a Critical Agent specializing in analyzing proposals and finding flaws.\nYour role in the swarm is to evaluate solutions proposed by other agents and identify potential issues.\nYou should carefully examine proposed solutions, find weaknesses or oversights, and suggest improvements.\nBe constructive in your criticism while ensuring the final solution is robust.\n""", \nname="critical_agent",model="us.anthropic.claude-3-7-sonnet-20250219-v1:0")summarizer_agent = Agent(system_prompt="""You are a Summarizer Agent specializing in synthesizing information.\nYour role in the swarm is to gather insights from all agents and create a cohesive final solution.\nYou should combine the best ideas and address the criticisms to create a comprehensive response.\nFocus on creating a clear, actionable summary that addresses the original query effectively.\n""",name="summarizer_agent",model="us.anthropic.claude-3-7-sonnet-20250219-v1:0")

エージェント調整は、Strands SDK のネイティブ Swarm クラスを使用して実装され、調整ツールを自動的に提供し共有コンテキストを管理します：

In [ ]:
from strands.multiagent import Swarm# Create a swarm with these agentsswarm = Swarm(    [research_agent, creative_agent, critical_agent, summarizer_agent],    max_handoffs=20,    max_iterations=20,    execution_timeout=900.0,  # 15 minutes    node_timeout=300.0,       # 5 minutes per agent    repetitive_handoff_detection_window=8,  # There must be >= 3 unique agents in the last 8 handoffs    repetitive_handoff_min_unique_agents=3)

swarm は、エージェントの専門知識とタスク要件に基づいていつハンドオフするかを決定する自律エージェント調整を通じて動作します：

In [ ]:
# Execute the swarm on a taskresult = swarm("Create a blog post explaining Agentic AI then create a summary for a social media post.")

In [ ]:
# Access the final resultprint(f"Status: {result.status}")

この例では：

* research_agent が Agentic AI に関する情報を収集して開始
* creative_agent に引き継いで魅力的なコンテンツと構造を開発
* critical_agent が正確性と完全性をレビュー
* 最後に summarizer_agent がすべての寄与を包括的なブログ投稿に統合
* 必要に応じてエージェントが複数回引き継ぎ可能、swarm が自動的に調整を管理

In [ ]:
# See which agents were involvedfor node in result.node_history:    print(f"Agent: {node.node_id}")# Get results from specific nodesresearch_result = result.results["research_agent"].resultprint(f"Research: {research_result}")# Get performance metricsprint(f"Total iterations: {result.execution_count}")print(f"Execution time: {result.execution_time}ms")print(f"Token usage: {result.accumulated_usage}")

swarm システムは、エージェント調整、共有コンテキスト、ハンドオフ機構を自動的に管理します。エージェントは他の専門家にいつ制御を移転するかを自律的に決定でき、タイムアウト保護や反復ハンドオフ検出などのビルトイン安全機能を備えたシームレスな協調ワークフローを実現します。


## Swarm を使用するタイミング：

- 多様な専門知識と専門知識を必要とする複雑なタスク
- 複数の視点と協調的問題解決が必要な場合
- 自律エージェント調整と集団知能の利点があるタスク
- テキストと画像などのマルチモーダル入力を一緒に扱う場合
- エージェント能力に基づく動的タスク配分が必要なプロジェクト

## ベストプラクティス

1. **専門化エージェントを作成**: Swarm の各エージェントに明確な役割を定義
2. **記述的なエージェント名を使用**: 名前はエージェントの専門性を反映
3. **適切なタイムアウトを設定**: タスクの複雑さと予想実行時間に基づいて調整
4. **反復ハンドオフ検出を有効化**: ピンポン動作を防ぐために `repetitive_handoff_detection_window` と `repetitive_handoff_min_unique_agents` に適切な値を設定
5. **多様な専門知識を含める**: Swarm に補完的なスキルのエージェントを確保
6. **エージェント説明を提供**: 他のエージェントがその能力を理解できるようにエージェントに説明を追加
7. **マルチモーダル入力を活用**: 画像を含むリッチな入力に ContentBlocks を使用

## 結論

マルチエージェント swarm は、出現集団知能を通じて複雑な問題を解決します。Strands Agents SDK は、柔軟な swarm ツールとカスタムエージェントチームを作成するためのネイティブ Swarm クラスを提供し、自律的に協力します。個別のモデルプロバイダ、ツール、設定を持つ専門化エージェントにタスクを分散させることで、単独で動作する単一エージェントよりも優れた結果を達成します。ビルトイン調整ツール、共有コンテキスト、包括的な安全機構により、開発者は自己組織化協調を通じて複雑で多面的な問題を扱う洗練されたマルチエージェントシステムを作成できます。

Swarm の詳細については [Strands ドキュメント](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/multi-agent/swarm/) をご覧ください。